In [4]:
pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 13.6 MB/s eta 0:00:00


In [5]:
from dgl.data import FB15k237Dataset

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [6]:
dataset = FB15k237Dataset()

Extracting file to /root/.dgl/FB15k-237_40695531
# entities: 14541
# relations: 237
# training edges: 272115
# validation edges: 17535
# testing edges: 20466
Done saving data into cached files.


In [18]:
dataset = FB15k237Dataset()
data = dataset[0]

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data):
      self.head, self.tail = data.edges()
      self.labels = data.edata['etype']


    def __len__(self):
        return len(self.head)

    def __getitem__(self, idx):
      head = torch.tensor(self.head[idx])
      tail = torch.tensor(self.tail[idx])
      label = torch.tensor(self.labels[idx])

      return head, label, tail

dataset = CustomDataset(data)

dataloader = DataLoader(
      dataset,
    batch_size = 64,
    shuffle = True,
)

In [14]:
# data에서 head, tail, label 추출
head, tail = data.edges()
label = data.edata['etype']


In [ ]:
import numpy as np
import torch.nn.init as init
import math
import torch.nn.functional as F

class TransE(nn.Module):
    def __init__(self, k = 50, gamma = 1, graphs = data, epochs = 1000):
        super(TransE, self).__init__()

        self.head, self.tail = data.edges()
        self.labels = data.edata['etype']
        self.k = k
        self.epochs = epochs
        self.num_entity = len(data['entities'])
        self.num_label = len(self.labels)

        bound = 6 / math.sqrt(self.k)

        # entity: 시작은 그냥하고 update 할 때마다 (loop 돌 때마다 normalize 해주면 됨)
        self.entity = nn.Parameter(init.uniform_(torch.empty(self.num_entity, self.k), -bound, bound), requires_grad=True)
        self.label = nn.Parameter(init.uniform_(torch.empty(self.num_label, self.k), -bound, bound), requires_grad=True)
        self.label.data = F.normalize(self.embed_label.data, p=2, dim=1)

        self.gamma = gamma

    def dissimiarity(self, h, l, t):
      # h, t: (k,) / l : (k, )
      d = torch.sum(torch.pow(h, 2)) + torch.sum(torch.pow(l, 2)) + torch.sum(torch.pow(t, 2)) - 2 * (torch.matmul(h, t) + torch.matmul(l, t - h))
      return d

    def forward(self, batch):
        head, label, tail = batch['head'], batch['label'], batch['tail']



    def fit(self):
      train_loss = []

      for epoch in range(self.epochs):
